In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import tensorflow as tf
#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


## Preprocessing

In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 1000
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = list(application_counts[application_counts < cutoff].index)
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
# Group low-frequency AFFILIATION categories
low_freq_categories = ['Family/Parent', 'National', 'Regional', 'Other']
application_df['AFFILIATION'] = application_df['AFFILIATION'].apply(lambda x: 'Other' if x in low_freq_categories else x)
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 1000
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = list(classification_counts[classification_counts < cutoff].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
# Group low-frequency USE_CASE categories
low_freq_categories = ['CommunityServ', 'Heathcare', 'Other']
application_df['USE_CASE'] = application_df['USE_CASE'].apply(lambda x: 'Other' if x in low_freq_categories else x)
# Group low-frequency ORGANIZATION categories
low_freq_categories = ['Co-operative', 'Corporation']
application_df['ORGANIZATION'] = application_df['ORGANIZATION'].apply(lambda x: 'Other' if x in low_freq_categories else x)
# Drop the STATUS Column
application_df = application_df.drop(['STATUS'], axis=1)
# Convert the INCOME_AMT column to a categorical variable
income_categories = ['0', '1M-5M', '100000-499999', '10M-50M', '25000-99999', '5M-10M', '10000-24999', '50M+']
application_df['INCOME_AMT'] = pd.Categorical(application_df['INCOME_AMT'], categories=income_categories, ordered=True)
# Replace missing values with the mode
income_mode = application_df['INCOME_AMT'].mode()[0]
application_df['INCOME_AMT'].replace(['1/01/9999'], income_mode, inplace=True)
# One-hot encode the INCOME_AMT column
income_dummies = pd.get_dummies(application_df['INCOME_AMT'], prefix='INCOME')
application_df = pd.concat([application_df, income_dummies], axis=1)
# Drop the original INCOME_AMT column
application_df.drop(['INCOME_AMT'], axis=1, inplace=True)
# Drop the SPECIAL_CONSIDERATIONS Column
application_df = application_df.drop(['SPECIAL_CONSIDERATIONS'], axis=1)
# Initialize the MinMaxScaler
scaler = MinMaxScaler()
# Reshape the feature to a 2D array and fit the scaler to the data
ask_amt = application_df["ASK_AMT"].values.reshape(-1, 1)
scaler.fit(ask_amt)
# Transform the feature using the scaler
scaled_ask_amt = scaler.transform(ask_amt)
# Replace the original feature in the DataFrame with the scaled feature
application_df["ASK_AMT"] = scaled_ask_amt
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['AFFILIATION','CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'APPLICATION_TYPE'])
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,INCOME_0,INCOME_1M-5M,INCOME_100000-499999,INCOME_10M-50M,INCOME_25000-99999,INCOME_5M-10M,INCOME_10000-24999,INCOME_50M+,...,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Other,ORGANIZATION_Trust,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6
0,0.000000e+00,1,1,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0
1,1.204843e-05,1,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,0.000000e+00,0,1,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,1,0
3,1.967945e-07,1,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
4,1.600293e-05,1,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0


## Split Preprocessed Data & Scale

In [3]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop('IS_SUCCESSFUL', axis=1).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

# Create a StandardScaler instances
scaler = StandardScaler()
# Fit the StandardScaler
X_scaler = scaler.fit(X_train)
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=30))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [8]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [9]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 56 Complete [00h 00m 31s]
val_accuracy: 0.7269970774650574

Best val_accuracy So Far: 0.7273469567298889
Total elapsed time: 00h 09m 52s
INFO:tensorflow:Oracle triggered exit


In [10]:
# Get top hyperparameter and print the values
top_hyper = tuner.get_best_hyperparameters(1)
for param in top_hyper:
    print(param.values)

{'activation': 'tanh', 'first_units': 9, 'num_layers': 3, 'units_0': 5, 'units_1': 7, 'units_2': 7, 'units_3': 1, 'units_4': 9, 'units_5': 1, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
